In [1]:
# TRANSFER LEARNING FINE TUNING -  MASS CALC BENIGN MALIGN NORMAL CLASSIFIER USING MAMMOGRAM PATCHES IN CBIS-DDSM
import numpy as np
import tensorflow.keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, Lambda, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, AveragePooling2D, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
#from tensorflow.keras.utils import np_utils
%matplotlib inline
tensorflow.keras.backend.set_image_data_format('channels_last')
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.layers import ReLU
import time


vid='v007'

In [2]:
# Create CNN

IMG_SIZE=224

from tensorflow.keras.applications.xception import Xception
base_model=Xception(weights='imagenet',include_top=False, input_shape=(IMG_SIZE,IMG_SIZE,3))

for layer in base_model.layers:
    layer.trainable=False

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(512)(x)
x=ReLU()(x)
x=Dropout(0.5)(x)
x=Dense(512)(x) 
x=ReLU()(x)
preds=Dense(5,activation='softmax')(x)

model=Model(inputs=base_model.input,outputs=preds)

In [3]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [4]:
len(model.layers)

139

In [5]:
from tensorflow.keras import optimizers
model.compile(optimizer=optimizers.Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy']) 

In [6]:
# Image preprocessing and data augmentation

batch_size=8

train_datagen = ImageDataGenerator(rescale=1./255,
                         horizontal_flip = True,
                         vertical_flip = True,
                         fill_mode="wrap",
                         width_shift_range = 0.2,
                         height_shift_range = 0.2,
                         zoom_range = 0.2,
                         rotation_range = 90,
                         shear_range=0.2
                        )

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory('CBIS_DDSM_PATCHES_01/train', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True)
valid_generator = valid_datagen.flow_from_directory('CBIS_DDSM_PATCHES_01/valid', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True)


Found 2170 images belonging to 5 classes.
Found 545 images belonging to 5 classes.


In [7]:
#Callbacks
#checkpoint_path="resnet50_train4/cp.ckpt"
def get_callbacks(name_weights, patience_lr):
    mcp_save = ModelCheckpoint(name_weights, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=2, min_delta=1e-4, mode='min')
    early_stop_cr=EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=2, mode='auto', restore_best_weights=True)
    return [mcp_save, reduce_lr_loss, early_stop_cr]


In [8]:
#Training stage 1
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size

name_weights = "mas_cal_ben_mal_class_resnet50_weights_"+vid+".h5"
callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=3,
                   verbose=1,
                   callbacks = callbacks)  

C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/3
271/271 [==============================] - 186s 579ms/step - loss: 1.3451 - accuracy: 0.4343 - val_loss: 1.0778 - val_accuracy: 0.5018


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/3
271/271 [==============================] - 48s 178ms/step - loss: 1.1279 - accuracy: 0.5185 - val_loss: 1.0101 - val_accuracy: 0.5662
Epoch 3/3
271/271 [==============================] - 49s 180ms/step - loss: 1.0868 - accuracy: 0.5241 - val_loss: 1.0068 - val_accuracy: 0.5662


In [9]:
#Training stage 2
for layer in model.layers[:90]:
    layer.trainable=False
for layer in model.layers[90:]:  #last ~33% layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=10,
                   verbose=1,
                   callbacks = callbacks)

Epoch 1/10
271/271 [==============================] - 59s 186ms/step - loss: 1.0629 - accuracy: 0.5513 - val_loss: 1.0112 - val_accuracy: 0.6176
Epoch 2/10
271/271 [==============================] - 50s 184ms/step - loss: 0.8804 - accuracy: 0.6341 - val_loss: 0.8448 - val_accuracy: 0.6176
Epoch 3/10
271/271 [==============================] - 48s 175ms/step - loss: 0.8036 - accuracy: 0.6688 - val_loss: 0.7235 - val_accuracy: 0.6765
Epoch 4/10
271/271 [==============================] - 49s 179ms/step - loss: 0.7490 - accuracy: 0.6795 - val_loss: 0.7956 - val_accuracy: 0.6544
Epoch 5/10
271/271 [==============================] - 48s 175ms/step - loss: 0.7379 - accuracy: 0.6906 - val_loss: 0.6718 - val_accuracy: 0.6985
Epoch 6/10
271/271 [==============================] - 46s 169ms/step - loss: 0.7219 - accuracy: 0.6970 - val_loss: 0.7477 - val_accuracy: 0.6783
Epoch 7/10
271/271 [==============================] - 45s 167ms/step - loss: 0.6723 - accuracy: 0.7095 - val_loss: 0.6355 - val_ac

In [10]:
#Training stage 3
#time.sleep(360)
for layer in model.layers:  #All layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
callbacks = get_callbacks(name_weights = name_weights, patience_lr=5)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=150,
                   verbose=1,
                   callbacks = callbacks)

Epoch 1/150
271/271 [==============================] - 53s 165ms/step - loss: 0.8999 - accuracy: 0.6286 - val_loss: 1.0243 - val_accuracy: 0.5956
Epoch 2/150
271/271 [==============================] - 44s 160ms/step - loss: 0.7901 - accuracy: 0.6758 - val_loss: 1.0678 - val_accuracy: 0.5901
Epoch 3/150
271/271 [==============================] - 43s 160ms/step - loss: 0.7168 - accuracy: 0.7081 - val_loss: 0.6966 - val_accuracy: 0.7132
Epoch 4/150
271/271 [==============================] - 44s 161ms/step - loss: 0.7115 - accuracy: 0.7068 - val_loss: 0.6788 - val_accuracy: 0.7151
Epoch 5/150
271/271 [==============================] - 46s 170ms/step - loss: 0.6828 - accuracy: 0.7081 - val_loss: 0.6845 - val_accuracy: 0.7132
Epoch 6/150
271/271 [==============================] - 46s 169ms/step - loss: 0.6713 - accuracy: 0.7100 - val_loss: 0.6773 - val_accuracy: 0.7188
Epoch 7/150
271/271 [==============================] - 48s 178ms/step - loss: 0.6488 - accuracy: 0.7294 - val_loss: 0.6981 -

In [11]:
model.metrics_names

['loss', 'accuracy']

In [12]:
# Validation accuracy, validation data confusion matrix, area under the roc score
valid_generator = valid_datagen.flow_from_directory('CBIS_DDSM_PATCHES_01/valid', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=False)

# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict_generator(valid_generator)
y_pred = np.argmax(Y_pred, axis=1)
ras=roc_auc_score(valid_generator.classes, Y_pred,multi_class='ovr')
#prec=precision_score(valid_generator.classes, y_pred,average='samples')
#rec=recall_score(valid_generator.classes, y_pred,average='samples')
#f1s=f1_score(valid_generator.classes, y_pred,average='samples')


print('Confusion Matrix:')
print(confusion_matrix(valid_generator.classes, y_pred))
print('Accuracy:', accuracy_score(valid_generator.classes, y_pred))
print('ROC AUC score:', ras)
#print('Precision:', prec)
#print('Recall:', rec)
#print('f1score',f1s)

Found 545 images belonging to 5 classes.


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


Confusion Matrix:
[[ 77  22   2   7   1]
 [ 17  68   6  18   0]
 [  3   1  52  52   1]
 [  3   3  13  90   0]
 [  2   0   1   1 105]]
Accuracy: 0.7192660550458716
ROC AUC score: 0.9406573520747413


In [13]:
train_generator.filenames

['calc_ben\\calc_ben_0001.png',
 'calc_ben\\calc_ben_0002.png',
 'calc_ben\\calc_ben_0003.png',
 'calc_ben\\calc_ben_0004.png',
 'calc_ben\\calc_ben_0006.png',
 'calc_ben\\calc_ben_0007.png',
 'calc_ben\\calc_ben_0008.png',
 'calc_ben\\calc_ben_0009.png',
 'calc_ben\\calc_ben_0017.png',
 'calc_ben\\calc_ben_0019.png',
 'calc_ben\\calc_ben_0020.png',
 'calc_ben\\calc_ben_0022.png',
 'calc_ben\\calc_ben_0023.png',
 'calc_ben\\calc_ben_0024.png',
 'calc_ben\\calc_ben_0025.png',
 'calc_ben\\calc_ben_0034.png',
 'calc_ben\\calc_ben_0035.png',
 'calc_ben\\calc_ben_0036.png',
 'calc_ben\\calc_ben_0038.png',
 'calc_ben\\calc_ben_0041.png',
 'calc_ben\\calc_ben_0049.png',
 'calc_ben\\calc_ben_0050.png',
 'calc_ben\\calc_ben_0051.png',
 'calc_ben\\calc_ben_0053.png',
 'calc_ben\\calc_ben_0054.png',
 'calc_ben\\calc_ben_0056.png',
 'calc_ben\\calc_ben_0057.png',
 'calc_ben\\calc_ben_0065.png',
 'calc_ben\\calc_ben_0067.png',
 'calc_ben\\calc_ben_0069.png',
 'calc_ben\\calc_ben_0071.png',
 'calc_b

In [14]:
valid_generator.filenames

['calc_ben\\calc_ben_0005.png',
 'calc_ben\\calc_ben_0018.png',
 'calc_ben\\calc_ben_0021.png',
 'calc_ben\\calc_ben_0033.png',
 'calc_ben\\calc_ben_0037.png',
 'calc_ben\\calc_ben_0039.png',
 'calc_ben\\calc_ben_0040.png',
 'calc_ben\\calc_ben_0052.png',
 'calc_ben\\calc_ben_0055.png',
 'calc_ben\\calc_ben_0066.png',
 'calc_ben\\calc_ben_0068.png',
 'calc_ben\\calc_ben_0070.png',
 'calc_ben\\calc_ben_0072.png',
 'calc_ben\\calc_ben_0081.png',
 'calc_ben\\calc_ben_0082.png',
 'calc_ben\\calc_ben_0089.png',
 'calc_ben\\calc_ben_0098.png',
 'calc_ben\\calc_ben_0102.png',
 'calc_ben\\calc_ben_0114.png',
 'calc_ben\\calc_ben_0115.png',
 'calc_ben\\calc_ben_0129.png',
 'calc_ben\\calc_ben_0137.png',
 'calc_ben\\calc_ben_0146.png',
 'calc_ben\\calc_ben_0147.png',
 'calc_ben\\calc_ben_0149.png',
 'calc_ben\\calc_ben_0150.png',
 'calc_ben\\calc_ben_0152.png',
 'calc_ben\\calc_ben_0166.png',
 'calc_ben\\calc_ben_0167.png',
 'calc_ben\\calc_ben_0184.png',
 'calc_ben\\calc_ben_0193.png',
 'calc_b

In [15]:
#Print estimation result for ensemble model
print(Y_pred)
print(valid_generator.classes)

[[9.9962366e-01 3.6807233e-04 3.4470752e-06 4.7270487e-06 8.8632990e-10]
 [9.6846157e-01 8.1112105e-03 8.8846628e-03 1.4508101e-02 3.4356606e-05]
 [3.5027429e-01 5.1057428e-01 7.1107976e-02 6.5448485e-02 2.5949250e-03]
 ...
 [2.2080765e-04 3.4216352e-04 5.3254591e-04 4.1402996e-04 9.9849045e-01]
 [1.2290612e-04 1.4790273e-04 8.7488675e-03 4.0203976e-03 9.8695999e-01]
 [3.7482223e-06 4.5516372e-06 2.9582345e-05 1.3105997e-05 9.9994898e-01]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 

In [16]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory('CBIS_DDSM_PATCHES_01/test', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=False)


# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict_generator(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
ras=roc_auc_score(test_generator.classes, Y_pred,multi_class='ovr')


print('Confusion Matrix:')
print(confusion_matrix(test_generator.classes, y_pred))
print('Accuracy:', accuracy_score(test_generator.classes, y_pred))
print('ROC AUC score:', ras)


Found 645 images belonging to 5 classes.


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


Confusion Matrix:
[[ 69  41   4  14   1]
 [ 19  84   8  17   1]
 [  2   1  67  58   1]
 [  2   4  13 110   0]
 [  1   0   3   0 125]]
Accuracy: 0.7054263565891473
ROC AUC score: 0.9316928069226609


In [17]:
test_generator.filenames

['calc_ben\\calc_ben_1003.png',
 'calc_ben\\calc_ben_1004.png',
 'calc_ben\\calc_ben_1005.png',
 'calc_ben\\calc_ben_1006.png',
 'calc_ben\\calc_ben_1007.png',
 'calc_ben\\calc_ben_1008.png',
 'calc_ben\\calc_ben_1009.png',
 'calc_ben\\calc_ben_1010.png',
 'calc_ben\\calc_ben_1011.png',
 'calc_ben\\calc_ben_1012.png',
 'calc_ben\\calc_ben_1013.png',
 'calc_ben\\calc_ben_1019.png',
 'calc_ben\\calc_ben_1020.png',
 'calc_ben\\calc_ben_1021.png',
 'calc_ben\\calc_ben_1022.png',
 'calc_ben\\calc_ben_1023.png',
 'calc_ben\\calc_ben_1024.png',
 'calc_ben\\calc_ben_1025.png',
 'calc_ben\\calc_ben_1026.png',
 'calc_ben\\calc_ben_1027.png',
 'calc_ben\\calc_ben_1028.png',
 'calc_ben\\calc_ben_1029.png',
 'calc_ben\\calc_ben_1035.png',
 'calc_ben\\calc_ben_1036.png',
 'calc_ben\\calc_ben_1037.png',
 'calc_ben\\calc_ben_1039.png',
 'calc_ben\\calc_ben_1040.png',
 'calc_ben\\calc_ben_1041.png',
 'calc_ben\\calc_ben_1042.png',
 'calc_ben\\calc_ben_1043.png',
 'calc_ben\\calc_ben_1044.png',
 'calc_b

In [20]:
#Print estimation result
print(Y_pred)
print(test_generator.classes)
len(test_generator.classes)

[[6.7295891e-01 2.0795082e-01 7.1409337e-02 4.4860166e-02 2.8206939e-03]
 [1.0447173e-01 1.7082919e-01 2.2861111e-01 4.9406841e-01 2.0195069e-03]
 [7.6823837e-01 1.6907872e-01 4.2092107e-02 2.0442778e-02 1.4795984e-04]
 ...
 [1.3144560e-02 1.7186085e-02 4.2238902e-02 4.1011091e-02 8.8641930e-01]
 [3.7869604e-04 4.4688990e-04 1.9319558e-03 7.6256116e-04 9.9647993e-01]
 [3.8969881e-04 5.4597558e-04 4.4166301e-03 1.7488598e-03 9.9289882e-01]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 

645